<a href="https://colab.research.google.com/github/MINNUZ17/house_price_prediction/blob/main/Another_copy_of_House_price_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Load the csv file

In [ ]:
df = pd.read_csv("/content/Bangalore  house data.csv")

In [ ]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [ ]:
df.shape

(13320, 9)

In [ ]:
df.isnull().sum()

,0
area_type,0
availability,0
location,1
size,16
society,5502
total_sqft,0
bath,73
balcony,609
price,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


In [ ]:
# Fill missing values in 'bath' and 'balcony' with median
df['bath'].fillna(df['bath'].median(), inplace=True)
df['balcony'].fillna(df['balcony'].median(), inplace=True)

# Drop rows with missing values in 'location' or 'size'
df.dropna(subset=['location', 'size'], inplace=True)


<ipython-input-7-1299716c67b0>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bath'].fillna(df['bath'].median(), inplace=True)
<ipython-input-7-1299716c67b0>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [ ]:
df.describe()

,bath,balcony,price
count,13303.000000,13303.000000,13303.000000
mean,2.689619,1.602872,112.584033
std,1.339393,0.803455,148.993820
min,1.000000,0.000000,8.000000
25%,2.000000,1.000000,50.000000
50%,2.000000,2.000000,72.000000
75%,3.000000,2.000000,120.000000
max,40.000000,3.000000,3600.000000


In [ ]:
df_house = df.drop(columns=['area_type', 'availability', 'society'])

Data Preprocessing

In [ ]:
df_house.columns

Index(['location', 'size', 'total_sqft', 'bath', 'balcony', 'price'], dtype='object')

In [ ]:
df_house.head()

,location,size,total_sqft,bath,balcony,price
0,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Kothanur,2 BHK,1200,2.0,1.0,51.00


In [ ]:
def convert_sqft_to_num(x):
    try:
        if '-' in x:
            a, b = x.split('-')
            return (float(a) + float(b)) / 2
        else:
            return float(x)
    except:
        return np.nan

df_house['total_sqft'] = df_house['total_sqft'].apply(convert_sqft_to_num)
df_house.dropna(inplace=True)


In [ ]:
# Split the dataset into dependent(y) & independent(X) variables,
# Dependent variable is also called the target variable which is price of the car in our case

In [ ]:
X=df_house.drop(columns=["price"])
y=df_house["price"]

In [ ]:
# Specify two empty lists cat_col & num_col to store categorical and numerical columns respectively

In [ ]:
cat_col=[]#will store categorical features
num_col=[]#will store numerical features

#iterating thourgh all columns in X
for col in X:
    #append the features whose datatype is object in cat_col
    if df_house[col].dtype=="O":
        cat_col.append(col)
    #append those features whose datatype is other than object in num_col
    else:
        num_col.append(col)

In [ ]:
cat_col

['location', 'size']

In [ ]:
num_col

['total_sqft', 'bath', 'balcony']

In [ ]:
# Create dataFrames df_cat & df_num to store the features with datatypes object and
# numerical respectively

# By dividing the dataframe into numerical & categorical features seperately,
# it will allow an ease handling of numerical and categorical features in
# their respective dataframes.

In [ ]:
#dataframe to store the categorical features
df_cat=pd.DataFrame(
    data=df_house,
    #we will use the column names from the cat_col list
    columns=cat_col
)

#dataframe to store the categorical features
df_num=pd.DataFrame(
    data=df_house,
    #we will use the column names from the num_col list
    columns=num_col
)

In [ ]:
df_cat.head()

,location,size
0,Electronic City Phase II,2 BHK
1,Chikka Tirupathi,4 Bedroom
2,Uttarahalli,3 BHK
3,Lingadheeranahalli,3 BHK
4,Kothanur,2 BHK


In [ ]:
df_num.head()

,total_sqft,bath,balcony
0,1056.0,2.0,1.0
1,2600.0,5.0,3.0
2,1440.0,2.0,3.0
3,1521.0,3.0,1.0
4,1200.0,2.0,1.0


In [ ]:
# Print the labels of each columns in df_cat

In [ ]:
for cols in df_cat:
    print(cols," contains :",df_cat[cols].nunique()," labels")

location  contains : 1298  labels
size  contains : 31  labels


In [ ]:
# one-hot encoding

In [ ]:
df_cat = pd.get_dummies(df_cat, drop_first=True)

In [ ]:
df_cat.head()

,location_ Banaswadi,location_ Basavangudi,location_ Bhoganhalli,location_ Devarabeesana Halli,location_ Devarachikkanahalli,location_ Electronic City,location_ Mysore Highway,location_ Rachenahalli,location_ Sector 1 HSR Layout,location_ Thanisandra,...,size_5 BHK,size_5 Bedroom,size_6 BHK,size_6 Bedroom,size_7 BHK,size_7 Bedroom,size_8 BHK,size_8 Bedroom,size_9 BHK,size_9 Bedroom
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# combining categorical and numerical data

In [ ]:
house_final=pd.concat(
    [df_num,df_cat,y],
    axis=1
)

In [ ]:
house_final.head()

,total_sqft,bath,balcony,location_ Banaswadi,location_ Basavangudi,location_ Bhoganhalli,location_ Devarabeesana Halli,location_ Devarachikkanahalli,location_ Electronic City,location_ Mysore Highway,...,size_5 Bedroom,size_6 BHK,size_6 Bedroom,size_7 BHK,size_7 Bedroom,size_8 BHK,size_8 Bedroom,size_9 BHK,size_9 Bedroom,price
0,1056.0,2.0,1.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,39.07
1,2600.0,5.0,3.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,120.00
2,1440.0,2.0,3.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,62.00
3,1521.0,3.0,1.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,95.00
4,1200.0,2.0,1.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,51.00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = house_final.drop('price', axis=1)
y = house_final['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# performing standardization

# Model training and testing

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

Support Vector Regressor

Linear Regression

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

y_pred_lr = lr_model.predict(X_test_scaled)

print("Linear Regression:")
print("R² Score:", r2_score(y_test, y_pred_lr))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lr)))
print()

Linear Regression:
R² Score: 0.4907237981927709
RMSE: 105.48631320126455



Decision Tree Regression

In [ ]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train_scaled, y_train)

y_pred_dt = dt_model.predict(X_test_scaled)

print("Decision Tree Regression:")
print("R² Score:", r2_score(y_test, y_pred_dt))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_dt)))


Decision Tree Regression:
R² Score: 0.5624318710239276
RMSE: 97.77825077192722


Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

In [ ]:
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)

y_pred_rf = rf_model.predict(X_test_scaled)

print("Random Forest Regression:")
print("R² Score:", r2_score(y_test, y_pred_rf))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))

Random Forest Regression:
R² Score: 0.6512119103045694
RMSE: 87.29718624964235


In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train_scaled, y_train)
y_pred_xgb = xgb_model.predict(X_test_scaled)

print("XGBoost:")
print("R² Score:", r2_score(y_test, y_pred_xgb))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_xgb)))


XGBoost:
R² Score: 0.6540026458817614
RMSE: 86.9472421832807


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gbr.fit(X_train_scaled, y_train)
y_pred_gbr = gbr.predict(X_test_scaled)

print("Gradient Boosting:")
print("R² Score:", r2_score(y_test, y_pred_gbr))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_gbr)))


Gradient Boosting:
R² Score: 0.6616675499583831
RMSE: 85.97877427935032


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

et_model = ExtraTreesRegressor(n_estimators=100, random_state=42)
et_model.fit(X_train_scaled, y_train)
y_pred_et = et_model.predict(X_test_scaled)

print("Extra Trees:")
print("R² Score:", r2_score(y_test, y_pred_et))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_et)))


Extra Trees:
R² Score: 0.7058467915951947
RMSE: 80.16895405283829


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.5 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

model = CatBoostRegressor(verbose=0)  # quiet mode
model.fit(X_train_scaled, y_train)

y_pred_cb = model.predict(X_test_scaled)

print("CatBoost:")
print("R² Score:", r2_score(y_test, y_pred_cb))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_cb)))


CatBoost:
R² Score: 0.6648187353414248
RMSE: 85.57743991799438
